# Preparação

## Imports

In [1]:

import os

from pathlib import Path

import duckdb
import pandas as pd

from dotenv import load_dotenv

load_dotenv()

PROJECT_DIR = Path("~/tramita").expanduser()
DB_PATH = PROJECT_DIR / os.getenv("SILVER_DUCKDB_PATH", "")
OUT_DIR = PROJECT_DIR / "data" / "gold"
OUT_DIR.mkdir(exist_ok=True)

NODES_PATH_PARQUET = OUT_DIR / "nodes.parquet"
EDGES_PATH_PARQUET = OUT_DIR / "edges.parquet"
NODES_PATH_CSV = OUT_DIR / "nodes.csv"
EDGES_PATH_CSV = OUT_DIR / "edges.csv"

## Extração de nós e arestas

In [2]:
with duckdb.connect(DB_PATH, read_only=True) as con:

    house_props_df = con.execute("SELECT * FROM proposicoes_camara").df()
    house_autores_df = con.execute("SELECT * FROM autores_camara").df()
    house_deputados_df = con.execute("SELECT * FROM deputados_camara").df()
    house_orgaos_df = con.execute("SELECT * FROM orgaos_camara").df()
    
    senate_procs_df = con.execute("SELECT * FROM processo_senado").df()
    senate_autores_df = con.execute("SELECT * FROM autoria_iniciativa_senado").df()
    senate_parlamentares_df = con.execute("SELECT * FROM parlamentar_senado").df()
    senate_entes_df = con.execute("SELECT * FROM ente_senado").df()
    
    bill_match_df = con.execute("SELECT * FROM correspondencia_proposicoes_processo").df()

senate_parlamentares_df['tag'] = 'SS:' + senate_parlamentares_df['codigo_parlamentar'].astype(str)
senate_parlamentares_df

,codigo_parlamentar,codigo_publico_leg_atual,nome_completo,nome_parlamentar,sexo_parlamentar,sigla_partido,uf_parlamentar,email_parlamentar,data_nascimento,endereco_parlamentar,naturalidade,uf_naturalidade,year_snapshot,rn,tag
0,4606,<NA>,José Eleonildo Soares,Pinto Itamaraty,Masculino,PSDB,None,pinto.itamaraty@senador.leg.br,1960-05-10,Senado Federal Anexo I 25º Andar,São Luís,MA,2023,1,SS:4606
1,4811,900,Laércio José de Oliveira,Laércio Oliveira,Masculino,PP,SE,sen.laerciooliveira@senado.leg.br,1959-04-15,Senado Federal Anexo 2 Ala Teotônio Vilela G...,Recife,PE,2024,1,SS:4811
2,5257,<NA>,Renzo do Amaral Braz,Renzo Braz,Masculino,PP,None,None,1980-04-11,None,Muriaé,MG,2024,1,SS:5257
3,5537,928,Dário Elias Berger,Dário Berger,Masculino,PSB,None,sen.darioberger@senado.leg.br,1956-12-07,Senado Federal Anexo 1 16º Pavimento,Bom Retiro,SC,2023,1,SS:5537
4,5615,<NA>,Gilberto Piselo do Nascimento,Gilberto Piselo,Masculino,PDT,None,None,1959-07-04,None,None,None,2023,1,SS:5615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,6295,945,Carlos Henrique Baqueta Fávaro,Carlos Fávaro,Masculino,PSD,None,sen.carlosfavaro@senado.leg.br,1969-10-19,Senado Federal Anexo 2 Ala Teotônio Vilela G...,None,None,2024,1,SS:6295
314,6365,<NA>,Leny May da Silva Campêlo,Leny Campêlo,Feminino,None,None,None,1962-10-18,None,None,None,2024,1,SS:6365
315,6368,<NA>,Francisco Ferreira Alexandre,Francisco Alexandre,Masculino,None,None,None,1962-10-29,None,None,None,2024,1,SS:6368
316,6374,938,Flávio José Cavalcanti de Azevedo,Flavio Azevedo,Masculino,PL,None,sen.flavioazevedo@senado.leg.br,1946-02-19,Senado Federal Anexo 2 Ala Teotônio Vilela G...,None,None,2024,1,SS:6374


## Filtragem

In [3]:
# Na fase silver já havíamos eliminado as proposições anteriores a 2019.

# Agora vamos eliminar as autorias da câmara e do senado que não estejam contempladas nas proposições e processos que temos.

house_autores_df = house_autores_df[house_autores_df['id_proposicao'].isin(house_props_df['id_proposicao'])]
senate_autores_df = senate_autores_df[senate_autores_df['id_processo'].isin(senate_procs_df['id_processo'])]

In [4]:
# Da mesma forma, podemos eliminar deputados e órgãos e senadores não contemplados nas autorias

house_deputados_df = house_deputados_df[house_deputados_df['id_deputado'].isin(
    house_autores_df[house_autores_df['tipo_autor'] == 'deputados']['id_deputado_ou_orgao'].unique()
)]


house_orgaos_df = house_orgaos_df[house_orgaos_df['id_orgao'].isin(
    house_autores_df[house_autores_df['tipo_autor'] == 'orgaos']['id_deputado_ou_orgao'].unique()
)]

senate_parlamentares_df = senate_parlamentares_df[senate_parlamentares_df['codigo_parlamentar'].isin(
    senate_autores_df['codigo_parlamentar'].unique()
)]

In [5]:
missing_entes_df = senate_autores_df[senate_autores_df['sigla_ente'].isnull()][['ente', 'sigla_tipo', 'descricao_tipo']].drop_duplicates()
missing_entes_df

,ente,sigla_tipo,descricao_tipo
7772,Superior Tribunal de Justiça,TRIBUNAL_SUPERIOR,TRIBUNAL_SUPERIOR
7844,Procuradoria-Geral da República,PROCURADOR_GERAL,PROCURADOR_GERAL
19296,Ministério Público da União,MINISTERIO_PUBLICO_UNIAO,MINISTERIO_PUBLICO_UNIAO
20161,Comissão de Turismo,COMISSAO_CAMARA,COMISSAO_CAMARA
21462,Defensoria Pública da União,DEFENSOR_GERAL,DEFENSOR_GERAL
25415,Comissão especial destinada a acompanhar as aç...,COMISSAO_CAMARA,COMISSAO_CAMARA


In [6]:
ids_e_siglas = {7772:'_STJ', 7844:'_PGR', 19296:'_MPU', 20161:'_CdT', 21462:'_DPU', 25415:'_COMISSAO_CANCER'}
# para cada uma das autorias faltantes, vamos criar uma nova linha em senate_entes_df
k = 0
new_rows = []
for index, row in missing_entes_df.iterrows():
    sigla = ids_e_siglas.get(index)
    new_row = {
        'id_ente': 9999990 + k,
        'sigla': sigla,
        'nome': row['ente'],
        'casa': None,
        'sigla_tipo': row['sigla_tipo'],
        'descricao_tipo': row['descricao_tipo'],
        'data_inicio': None,
        'data_fim': None,
        'tag': f'SE:{9999990 + k}'
    }
    new_rows.append(new_row)
    k += 1
senate_entes_df = pd.concat([senate_entes_df, pd.DataFrame(new_rows)], ignore_index=True)
    

/tmp/ipykernel_1814655/320498163.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  senate_entes_df = pd.concat([senate_entes_df, pd.DataFrame(new_rows)], ignore_index=True)


In [7]:
# agora preenchemos as siglas faltantes em senate_autores_df

for index, row in missing_entes_df.iterrows():
    sigla = ids_e_siglas.get(index)
    senate_autores_df.loc[
        (senate_autores_df['ente'] == row['ente']) &
        (senate_autores_df['sigla_ente'].isnull()),
        'sigla_ente'
    ] = sigla

In [8]:
senate_entes_df[senate_entes_df['sigla'] == 'SF']

,id_ente,sigla,nome,casa,sigla_tipo,descricao_tipo,data_inicio,data_fim,tag
10861,1,SF,Senado Federal,SF,CASA_LEGISLATIVA,Casa Legislativa,1960-01-01,NaT,SE:1


In [9]:
# Agora fazemos o caminho inverso. Mantemos em senate_entes_df somente o que aparece em senate_autores_df
# ou seja, somente onde senate_entes_df.sigla está em senate_autores_df.sigla_ente
senate_entes_df = senate_entes_df[senate_entes_df['sigla'].isin(
    senate_autores_df['sigla_ente'].unique()
)]

# Se houver duplicadas em sigla_ente, mantemos a primeira ocorrência
senate_entes_df = senate_entes_df.drop_duplicates(subset=['sigla'], keep='first')

In [10]:
senate_entes_lookup_df = senate_entes_df[['sigla', 'id_ente']].set_index('sigla', drop=True)
senate_entes_lookup_df

,id_ente
sigla,
CPIPANDEMIA,7352398
CDIR,55226
CDH,3947422
CMA,3927825
CD,2
CBHS,7352682
Mesa,7349502
PR,55126
SF,1


In [11]:
# finalmente, criamos a coluna id_ente em senate_autores_df a partir de sigla_ente usando senate_entes_lookup_df
senate_autores_df['id_ente'] = senate_autores_df['sigla_ente'].map(senate_entes_lookup_df['id_ente'])

### Definições para nós e arestas

| Object          | Source df    | Columns to include   | Type        |
| ---             | ---          | ---                  |  ---        |
| Prop nodes      | house_props_df     | prop_tag, prop_label | Proposicao  |
| Deputados nodes | house_deputados_df | tag, nome_civil | Deputado |
| Órgãos nodes    | house_orgaos_df    | tag, nome | Orgao |
| Procs nodes     | senate_procs_df  | tag, identificacao | Processo |
| Parlamentares nodes | senate_parlamentares_df | codigo_parlamentar (with "SP:" prepended), nome_parlamentar | Senador |
| Entes nodes | senate_entes_df | tag, nome | Ente |

In [12]:
nodes_df = pd.concat([
    house_props_df[["prop_tag", "prop_label"]].rename(columns={"prop_tag": "tag", "prop_label": "label"}),
    house_deputados_df[["tag", "nome_civil"]].rename(columns={"nome_civil": "label"}),
    house_orgaos_df[["tag", "nome"]].rename(columns={"nome": "label"}),
    senate_procs_df[["tag", "identificacao"]].rename(columns={"identificacao": "label"}),
    senate_parlamentares_df[["tag", "nome_parlamentar"]].rename(columns={"nome_parlamentar": "label"}),
    senate_entes_df[["tag", "nome"]].rename(columns={"nome": "label"}),
], ignore_index=True).drop_duplicates().reset_index(drop=True)

def get_node_type(tag: str) -> str:
    prefix = tag[:3]
    match prefix:
        case 'CP:':
            return 'Proposicao'
        case 'CD:':
            return 'Deputado'
        case 'CO:':
            return 'Orgao'
        case 'SP:':
            return 'Processo'
        case 'SS:':
            return 'Senador'
        case 'SE:':
            return 'Ente'
        case _:
            return 'Unknown'
    
nodes_df['type'] = nodes_df['tag'].apply(get_node_type)
nodes_df.to_parquet(NODES_PATH_PARQUET, index=False)
nodes_df.to_csv(NODES_PATH_CSV, index=False)

In [13]:
def get_senate_auth_tag(row):
    if row['sigla_ente'] == 'SF' and not pd.isna(row['codigo_parlamentar']):
        return f"SS:{row['codigo_parlamentar']}"
    else:
        return f"SE:{row['id_ente']}"
        
senate_autores_df['proc_tag'] = senate_autores_df['id_processo'].apply(lambda x: f"SP:{x}")
senate_autores_df['auth_tag'] = senate_autores_df.apply(get_senate_auth_tag, axis=1)


In [14]:
senate_autores_df

,id_autoria_iniciativa,id_processo,codigo_parlamentar,descricao_tipo,ente,ordem,outros_autores_nao_informados,sigla_ente,sigla_tipo,year_snapshot,id_ente,proc_tag,auth_tag
1954,1955,7711601,5627,SENADOR,Senado Federal,2,Não,SF,SENADOR,2019,1,SP:7711601,SS:5627
1955,1956,7711690,5411,SENADOR,Senado Federal,29,Não,SF,SENADOR,2019,1,SP:7711690,SS:5411
1956,1957,7712043,5976,SENADOR,Senado Federal,1,Não,SF,SENADOR,2019,1,SP:7712043,SS:5976
1957,1958,7714029,945,SENADOR,Senado Federal,1,Não,SF,SENADOR,2019,1,SP:7714029,SS:945
1958,1959,7714041,945,SENADOR,Senado Federal,1,Não,SF,SENADOR,2019,1,SP:7714041,SS:945
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35694,35695,8730961,6335,SENADOR,Senado Federal,5,Não,SF,SENADOR,2024,1,SP:8730961,SS:6335
35704,35705,8730961,5502,SENADOR,Senado Federal,4,Não,SF,SENADOR,2024,1,SP:8730961,SS:5502
35714,35715,8730961,6341,SENADOR,Senado Federal,3,Não,SF,SENADOR,2024,1,SP:8730961,SS:6341
35724,35725,8730961,6009,SENADOR,Senado Federal,2,Não,SF,SENADOR,2024,1,SP:8730961,SS:6009


*Edge generation*

* For each row in house_autores_df:
    * If proponente = False, skip.
    * With id_proposicao, generate the prop_label as CP:id_proposicao
    * With id_deputado_ou_orgao, generate the auth_label as CD:id_deputado_ou_orgao if tipo_autor = 'deputados' or CO:id_deputado_ou_orgao if tipo_autor = 'orgaos'

In [15]:
house_edges_df = house_autores_df[house_autores_df['proponente']].copy()
house_edges_df['prop_label'] = 'CP:' + house_edges_df['id_proposicao'].astype(str)
house_edges_df['auth_label'] = house_edges_df.apply(
    lambda row: f"CD:{row['id_deputado_ou_orgao']}" if row['tipo_autor'] == 'deputados' else f"CO:{row['id_deputado_ou_orgao']}",
    axis=1
)
house_edges_df = house_edges_df[['auth_label', 'prop_label']].rename(columns={'auth_label': 'source', 'prop_label': 'target'})

In [16]:
senate_autores_df

,id_autoria_iniciativa,id_processo,codigo_parlamentar,descricao_tipo,ente,ordem,outros_autores_nao_informados,sigla_ente,sigla_tipo,year_snapshot,id_ente,proc_tag,auth_tag
1954,1955,7711601,5627,SENADOR,Senado Federal,2,Não,SF,SENADOR,2019,1,SP:7711601,SS:5627
1955,1956,7711690,5411,SENADOR,Senado Federal,29,Não,SF,SENADOR,2019,1,SP:7711690,SS:5411
1956,1957,7712043,5976,SENADOR,Senado Federal,1,Não,SF,SENADOR,2019,1,SP:7712043,SS:5976
1957,1958,7714029,945,SENADOR,Senado Federal,1,Não,SF,SENADOR,2019,1,SP:7714029,SS:945
1958,1959,7714041,945,SENADOR,Senado Federal,1,Não,SF,SENADOR,2019,1,SP:7714041,SS:945
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35694,35695,8730961,6335,SENADOR,Senado Federal,5,Não,SF,SENADOR,2024,1,SP:8730961,SS:6335
35704,35705,8730961,5502,SENADOR,Senado Federal,4,Não,SF,SENADOR,2024,1,SP:8730961,SS:5502
35714,35715,8730961,6341,SENADOR,Senado Federal,3,Não,SF,SENADOR,2024,1,SP:8730961,SS:6341
35724,35725,8730961,6009,SENADOR,Senado Federal,2,Não,SF,SENADOR,2024,1,SP:8730961,SS:6009


In [17]:
senate_edges_df = senate_autores_df.copy()
senate_edges_df.rename(columns={'auth_tag': 'source', 'proc_tag': 'target'}, inplace=True)
senate_edges_df = senate_edges_df[['source', 'target']]
senate_edges_df

,source,target
1954,SS:5627,SP:7711601
1955,SS:5411,SP:7711690
1956,SS:5976,SP:7712043
1957,SS:945,SP:7714029
1958,SS:945,SP:7714041
...,...,...
35694,SS:6335,SP:8730961
35704,SS:5502,SP:8730961
35714,SS:6341,SP:8730961
35724,SS:6009,SP:8730961


In [18]:
edges_df = pd.concat([house_edges_df, senate_edges_df], ignore_index=True)
edges_df['etype'] = 'autoria'
edges_df

,source,target,etype
0,CD:160655,CP:538196,autoria
1,CD:141488,CP:559138,autoria
2,CD:73584,CP:593065,autoria
3,CD:160518,CP:601739,autoria
4,CD:151208,CP:614512,autoria
...,...,...,...
49837,SS:6335,SP:8730961,autoria
49838,SS:5502,SP:8730961,autoria
49839,SS:6341,SP:8730961,autoria
49840,SS:6009,SP:8730961,autoria


In [19]:
nodes_df

,tag,label,type
0,CP:2187087,PL 5029/2019,Proposicao
1,CP:2190408,PL 2/2019,Proposicao
2,CP:2190417,PL 10/2019,Proposicao
3,CP:2190423,PL 15/2019,Proposicao
4,CP:2190450,PL 21/2019,Proposicao
...,...,...,...
30942,SE:9999991,Procuradoria-Geral da República,Ente
30943,SE:9999992,Ministério Público da União,Ente
30944,SE:9999993,Comissão de Turismo,Ente
30945,SE:9999994,Defensoria Pública da União,Ente


In [20]:
bill_match_df['house_tag'] = 'CP:' + bill_match_df['id_proposicao_camara'].astype(str)
bill_match_df['senate_tag'] = 'SP:' + bill_match_df['id_processo_senado'].astype(str)
filtered_bill_match_df = bill_match_df[
    bill_match_df['house_tag'].isin(nodes_df[nodes_df['type'] == 'Proposicao']['tag']) &
    bill_match_df['senate_tag'].isin(nodes_df[nodes_df['type'] == 'Processo']['tag'])
]
filtered_bill_match_df = filtered_bill_match_df[['house_tag', 'senate_tag']].rename(columns={'house_tag': 'source', 'senate_tag': 'target'})
filtered_bill_match_df['etype'] = 'correspondencia'

In [21]:
edges_df = pd.concat([edges_df, filtered_bill_match_df], ignore_index=True)

In [22]:
edges_df.to_parquet(EDGES_PATH_PARQUET, index=False)
edges_df.to_csv(EDGES_PATH_CSV, index=False)

In [23]:
nodes_df.shape, edges_df.shape

((30947, 3), (51518, 3))

## Geração dos grafos

In [24]:
from igraph import Graph

In [25]:
edges_df = pd.read_csv(EDGES_PATH_CSV)
edges_df = edges_df.rename(columns={'source': 'from', 'target': 'to'})
edges_df.head()

,from,to,etype
0,CD:160655,CP:538196,autoria
1,CD:141488,CP:559138,autoria
2,CD:73584,CP:593065,autoria
3,CD:160518,CP:601739,autoria
4,CD:151208,CP:614512,autoria


In [26]:

nodes_df = pd.read_csv(NODES_PATH_CSV)
nodes_df = nodes_df.rename(columns={"tag": "name"})
nodes_df.head()

,name,label,type
0,CP:2187087,PL 5029/2019,Proposicao
1,CP:2190408,PL 2/2019,Proposicao
2,CP:2190417,PL 10/2019,Proposicao
3,CP:2190423,PL 15/2019,Proposicao
4,CP:2190450,PL 21/2019,Proposicao


In [27]:
edge_tuples = list(zip(edges_df['from'], edges_df['to']))
g = Graph.TupleList(
    edge_tuples,
    directed=False,
    vertex_name_attr="name"
)

In [28]:
for col in nodes_df.columns:
    if col != "name":
        g.vs[col] = nodes_df.set_index("name").loc[g.vs["name"], col].tolist()


In [29]:
for col in edges_df.columns:
    if col not in ("from", "to"):
        g.es[col] = edges_df[col].tolist()

In [30]:
print(g.summary())

IGRAPH UN-T 30941 51518 -- 
+ attr: label (v), name (v), type (v), etype (e)


In [31]:
summary = g.summary()
vcount, ecount = g.vcount(), g.ecount()
density = g.density()           # undirected density
components = g.components()     # undirected
component_sizes = pd.Series([len(c) for c in components], name="size").to_frame()
component_sizes["component_id"] = component_sizes.index
component_sizes = component_sizes[["component_id","size"]].sort_values("size", ascending=False).reset_index(drop=True)

In [32]:
comp_id_map = {}
for cid, comp in enumerate(components):
    for vid in comp:
        comp_id_map[vid] = cid

In [33]:
degree_all   = g.degree()                         # undirected degree
eigenvector  = g.eigenvector_centrality()         # undirected

In [34]:
community_method = None
try:
    cl = g.community_leiden(objective_function="modularity")
    community_method = "leiden"
except Exception:
    cl = g.community_multilevel()  # Louvain
    community_method = "louvain"

In [35]:
membership = cl.membership

In [36]:
try:
    proposicoes = set(name for name, t in zip(g.vs['name'], g.vs['type']) if t == 'Proposicao')

    corr_mask = [et == 'correspondencia' for et in g.es['etype']]
    corr_edges = [(g.vs[e.source]['name'], g.vs[e.target]['name'])
                  for e, m in zip(g.es, corr_mask) if m]

    # keep only edges where BOTH ends are proposições
    corr_edges = [(u, v) if (u in proposicoes and v in proposicoes) else None for (u, v) in corr_edges]
    corr_edges = [t for t in corr_edges if t is not None]

    if corr_edges:
        g_corr = Graph.TupleList(corr_edges, directed=False, vertex_name_attr="name")
        corr_components = g_corr.components()
        # Map proposição name -> correspondência group id
        corr_group = {}
        for gid, comp in enumerate(corr_components):
            for vid in comp:
                corr_group[g_corr.vs[vid]['name']] = gid
        # Assign per-vertex group id (non-proposições or isolated proposições get None)
        corr_group_id = []
        for nm, t in zip(g.vs['name'], g.vs['type']):
            if t == 'Proposicao' and nm in corr_group:
                corr_group_id.append(corr_group[nm])
            else:
                corr_group_id.append(None)
    else:
        corr_group_id = [None] * g.vcount()

except Exception:
    # If anything goes wrong, just leave as None
    corr_group_id = [None] * g.vcount()


In [41]:
vertex_metrics = pd.DataFrame({
    "id":                 g.vs["name"],
    "type":               g.vs["type"],
    "label": g.vs["label"],
    "degree":             degree_all,
    "eigenvector":        eigenvector,
    "community_id":       membership,
    "component_id":       [comp_id_map[i] for i in range(vcount)],
    "corr_group_id":      corr_group_id,   # same-bill groups (only for Proposicao; others = None)
})

In [42]:
def top_n(df: pd.DataFrame, col: str, n: int = 10, node_types: list | None = None) -> pd.DataFrame:
    sub = df if node_types is None else df[df["type"].isin(node_types)]
    return sub.sort_values(col, ascending=False).head(n).reset_index(drop=True)


In [47]:
top10_deps_eigen = top_n(vertex_metrics, "eigenvector", 10, node_types=["Deputado"])
top10_sens_eigen = top_n(vertex_metrics, "eigenvector", 10, node_types=["Senador"])
top10_orgs_eigen = top_n(vertex_metrics, "eigenvector", 10, node_types=["Orgao"])
top10_ents_eigen = top_n(vertex_metrics, "eigenvector", 10, node_types=["Ente"])


In [48]:
top10_deps_eigen

,id,type,label,degree,eigenvector,community_id,component_id,corr_group_id
0,CD:204467,Deputado,ROSA NEIDE SANDES DE ALMEIDA,235,1.000000,1,0,None
1,CD:178970,Deputado,JOAO SOMARIVA DANIEL,259,0.990990,1,0,None
2,CD:178986,Deputado,NILTO IGNACIO TATTO,253,0.986401,1,0,None
3,CD:74160,Deputado,PATRUS ANANIAS DE SOUZA,199,0.983884,1,0,None
4,CD:160535,Deputado,DIONILSO MATEUS MARCON,183,0.977833,1,0,None
5,CD:204393,Deputado,ANTONIO RIBEIRO,202,0.953757,1,0,None
6,CD:160610,Deputado,VALMIR CARLOS DA ASSUNÇÃO,195,0.950600,1,0,None
7,CD:204555,Deputado,JOSÉ RICARDO WENDLING,186,0.944690,1,0,None
8,CD:204480,Deputado,ROGÉRIO CORREIA DE MOURA BAPTISTA,191,0.938111,1,0,None
9,CD:204370,Deputado,CELIO ALVES DE MOURA,158,0.927276,1,0,None


In [49]:
top10_sens_eigen

,id,type,label,degree,eigenvector,community_id,component_id,corr_group_id
0,SS:345,Senador,Flávio Arns,77,9.404755e-12,66,0,None
1,SS:5748,Senador,Veneziano Vital do Rêgo,80,8.781748e-12,66,0,None
2,SS:825,Senador,Paulo Paim,112,7.986917e-12,66,0,None
3,SS:5895,Senador,Jorge Kajuru,79,7.159062e-12,66,0,None
4,SS:22,Senador,Esperidião Amin,62,6.506146e-12,66,0,None
5,SS:5985,Senador,Nelsinho Trad,57,6.346539e-12,66,0,None
6,SS:5979,Senador,Leila Barros,67,6.240429e-12,66,0,None
7,SS:1173,Senador,Wellington Fagundes,44,6.236565e-12,66,0,None
8,SS:5953,Senador,Fabiano Contarato,63,6.135934e-12,66,0,None
9,SS:475,Senador,Confúcio Moura,70,5.647549e-12,66,0,None


In [50]:
top10_orgs_eigen

,id,type,label,degree,eigenvector,community_id,component_id,corr_group_id
0,CO:78,Orgao,Senado Federal,893,6.301850e-08,66,0,None
1,CO:253,Orgao,Poder Executivo,589,0.000000e+00,65,6,None
2,CO:81,Orgao,Superior Tribunal de Justiça,3,0.000000e+00,90,12,None
3,CO:5438,Orgao,Comissão de Legislação Participativa,41,0.000000e+00,92,14,None
4,CO:101347,Orgao,Procuradoria-Geral da República,1,0.000000e+00,96,16,None
5,CO:2014,Orgao,Comissão de Saúde,8,0.000000e+00,100,19,None
6,CO:102210,Orgao,Comissão Mista da MPV 898/2019,1,0.000000e+00,101,20,None
7,CO:102212,Orgao,Comissão Mista da MPV 900/2019,1,0.000000e+00,102,21,None
8,CO:102211,Orgao,Comissão Mista da MPV 899/2019,1,0.000000e+00,104,23,None
9,CO:102220,Orgao,Comissão Mista da MPV 905/2019,1,0.000000e+00,107,25,None


In [51]:
top10_ents_eigen

,id,type,label,degree,eigenvector,community_id,component_id,corr_group_id
0,SE:2,Ente,Câmara dos Deputados,361,1.374919e-03,4,0,None
1,SE:7352398,Ente,CPI da Pandemia,5,5.967081e-13,66,0,None
2,SE:3947422,Ente,Comissão de Direitos Humanos e Legislação Part...,6,3.978775e-13,66,0,None
3,SE:3927825,Ente,Comissão de Meio Ambiente,2,3.975555e-13,66,0,None
4,SE:55226,Ente,Comissão Diretora,2,3.975555e-13,66,0,None
5,SE:1,Ente,Senado Federal,1,1.987350e-13,66,0,None
6,SE:55126,Ente,Presidência da República,613,0.000000e+00,65,6,None
7,SE:7352682,Ente,Forum Nacional de Comitês Hidrográficas Brasil,1,0.000000e+00,65,6,None
8,SE:55143,Ente,Tribunal de Justiça do Distrito Federal e Terr...,3,0.000000e+00,126,36,None
9,SE:9999991,Ente,Procuradoria-Geral da República,4,0.000000e+00,96,16,None


In [52]:
print(summary)
print(f"Vertices: {vcount}  Edges: {ecount}  Density: {density:.6f}")
print(f"Connected components: {len(components)}  (largest={component_sizes['size'].max()})")
print(f"Community method: {community_method}")
print("vertex_metrics columns:", list(vertex_metrics.columns))
print("Component sizes (head):\n", component_sizes.head())

IGRAPH UN-T 30941 51518 -- 
+ attr: label (v), name (v), type (v), etype (e)
Vertices: 30941  Edges: 51518  Density: 0.000108
Connected components: 110  (largest=28713)
Community method: leiden
vertex_metrics columns: ['id', 'type', 'label', 'degree', 'eigenvector', 'community_id', 'component_id', 'corr_group_id']
Component sizes (head):
    component_id   size
0             0  28713
1             6   1374
2             7     60
3            69     45
4            14     42
